In [19]:
import PIL
from PIL import Image
import torch
img = Image.open('Mask_Data/test/fully_covered/avgkcbjl.jpg')

In [20]:
img.mode

'L'

In [33]:
import numpy as np
import pandas as pd
import torch, torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvolutionBlock, self).__init__()

        self.convs = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=0, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(2),
            nn.ReLU()

        )

    def forward(self, X):
        return self.convs(X)

class Classifier(nn.Module):
    def __init__(self, in_channels,  num_classes) -> None:
        super(Classifier, self).__init__()

        conv1 = ConvolutionBlock(in_channels= in_channels, out_channels=8)
        conv2 = ConvolutionBlock(in_channels= 8, out_channels= 16)
        conv3 = ConvolutionBlock(in_channels= 16, out_channels=32)

        self.layers = nn.Sequential(conv1, conv2, conv3)

        self.fc = nn.Sequential(
            nn.Linear(in_features= 1152, out_features= 40),
            nn.ReLU(),
            nn.Linear(in_features=40, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=num_classes)
        )

    def forward(self, X):
        X = self.layers(X)
        X = torch.flatten(X)
        X = self.fc(X)

        return X

In [36]:
random = torch.rand(1,1,64,64)
trial_ = Classifier(1, 4)
output = trial_(random)
output

tensor([ 0.2705,  0.1012, -0.1437, -0.2186], grad_fn=<AddBackward0>)